# Predict expression from genotype

1. Load weights from trained LM and freeze
2. Take only the encoder part
3. Add a prediction head -> learnable params

In [44]:
import numpy as np
import pandas as pd
import pickle
import gc
import os
import re
import wandb

import sys
import pysam
import torch
from torch.utils.data import DataLoader, Dataset
from itertools import chain
from torch import nn

if not "../" in sys.path:
    sys.path.append("../")

from encoding_utils import sequence_encoders
import helpers.train_eval_a as train_eval    #train and evaluation
import helpers.misc as misc                #miscellaneous functions
from models.spec_dss import DSSResNetEmb, SpecAdd, DSSResNetExpression

from models.regression.train_eval import train_reg_model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the original model with checkpoints

In [35]:
import datetime
input_params = misc.dotdict({})

input_params.dataset =  '../../extdata/datasets/phase3_top10/dataset.parquet'
input_params.model_weight = '../../extdata/checkpoints/phase3_top10/aware_large_splitmsk/weights/epoch_100_weights_model.pt'


input_params.output_dir = '../../results/test'

input_params.split_mask = False
input_params.mask_rate = 0.2 #[0.012,0.2]#RAN #single float or 2 floats for reference and alternative
input_params.masking = 'none' # stratified_maf or none

input_params.diploid = True

input_params.test = False

input_params.get_embeddings = False
input_params.mask_at_test = True

input_params.agnostic = False

input_params.seq_len = 500

input_params.tot_epochs = 5
input_params.fold = 0
input_params.Nfolds = 5

input_params.train_splits = 1

input_params.save_at = range(1, input_params.tot_epochs)
input_params.validate_every = 1

input_params.d_model = 256
input_params.n_layers = 4
input_params.dropout = 0.

input_params.batch_size = 8
input_params.learning_rate = 1e-4
input_params.weight_decay = 0


input_params.haplotypes = False
input_params.run_name = "500_8_geno_full"

if input_params.run_name is None:
    input_params.run_name = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")




if input_params.dataset.endswith('.fa'):
    seq_df = pd.read_csv(input_params.dataset + '.fai', header=None, sep='\t', usecols=[0], names=['seq_name'])
elif input_params.dataset.endswith('.parquet'):
    seq_df = pd.read_parquet(input_params.dataset).reset_index()
    
seq_df[['split','sample_id','seg_name']] =  seq_df['seq_name'].str.split(':',expand=True)

if not input_params.agnostic:
    #for segment-aware model, assign a label to each segment
    seg_name = seq_df.seq_name.apply(lambda x:':'.join(x.split(':')[2:]))
    segment_encoding = seg_name.drop_duplicates().reset_index(drop=True)
    segment_encoding = {seg_name:idx for idx,seg_name in segment_encoding.items()}
    seq_df['seg_label'] = seg_name.map(segment_encoding)
else:
    seq_df['seg_label'] = 0


# get samples with expression data


# take the test set, it contains samples for which we have expression data
# seq_df = seq_df[seq_df.split=='test']

In [30]:

import datetim
run_name = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


In [31]:
run_name

'2024-01-05_16-24-33'

In [ ]:
df.loc[~df.index.isin(t)]

In [18]:
seq_expression_df

,seq_name,seq,split,sample_id,seg_name,seg_label,Gene_Symbol,expr
0,train:NA18870:ENSG00000179344.11,RRBRRBRBBBRRRRRRBRBBRBBRBRRRRRBBRBRBRRBRRRBBBB...,train,NA18870,ENSG00000179344.11,8,ENSG00000179344.11,57.33069
1,train:NA18870:ENSG00000179344.11,RRBRRBRBBBRRRRRRBRBBRBBRBRRRRRBBRBRBRRBRRRBBBB...,train,NA18870,ENSG00000179344.11,8,ENSG00000179344.11,68.38216
2,train:NA18508:ENSG00000196126.6,BRRRBBBRRRRBRRRBRBBBBBRRBBRRBBBRRRRBBRRBRBBBRB...,train,NA18508,ENSG00000196126.6,7,ENSG00000196126.6,862.92187
3,train:NA19114:ENSG00000196735.6,MMRMMMMRRRRRRRMMRMMRMMRRMRRMMMRRRRRMMMMMMMMMMM...,train,NA19114,ENSG00000196735.6,6,ENSG00000196735.6,386.36688
4,train:NA19114:ENSG00000196735.6,MMRMMMMRRRRRRRMMRMMRMMRRMRRMMMRRRRRMMMMMMMMMMM...,train,NA19114,ENSG00000196735.6,6,ENSG00000196735.6,535.41651
...,...,...,...,...,...,...,...,...
6375,test:NA20774:ENSG00000196735.6,BRMMMRMRMRMRMRRRRMMRMRRRRMMMMMRMRRMRMMMRMMMRBM...,test,NA20774,ENSG00000196735.6,6,ENSG00000196735.6,347.94405
6376,test:HG00267:ENSG00000232629.4,BBBBBBRBRBBBRBBRBBRBBRBBBBBRRRRRBRBBBBBBRRRBRB...,test,HG00267,ENSG00000232629.4,2,ENSG00000232629.4,7.41003
6377,test:HG00366:ENSG00000196735.6,BBRBBBBBBBBRBBBBRBBRBBRRBRRBBBRBRBRBBBBBBBBBBB...,test,HG00366,ENSG00000196735.6,6,ENSG00000196735.6,346.18428
6378,test:HG00379:ENSG00000176681.9,RRRRRRFRRRRRRFRRRRRRRRRRRRRRRRRRRRRRRRRRRRRBRR...,test,HG00379,ENSG00000176681.9,9,ENSG00000176681.9,5.83544


In [36]:
expression_data = pd.read_csv("../../extdata/GD660.GeneQuantRPKM.txt.gz", sep="\t")\
    .rename(columns=lambda x: re.sub("\..*",'',x))\
    .melt(id_vars = ["TargetID", "Gene_Symbol", "Chr", "Coord"], var_name = "sample_id", value_name = "expr")[["Gene_Symbol", "sample_id", "expr"]]
seq_expression_df = pd.merge(seq_df, expression_data, left_on=["sample_id", "seg_name"], right_on=["sample_id", "Gene_Symbol"])

# define test_set
test_indices = np.random.randint(0, len(seq_expression_df), size=int(0.1*len(seq_expression_df)))
save_path = os.path.join(
    input_params.output_dir, 
    input_params.run_name, 
    "test_set_indices.pkl"
)

if not os.path.exists(os.path.dirname(save_path)): 
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, "wb") as f:
    pickle.dump(test_indices, f)

seq_expression_df = seq_expression_df.loc[~seq_expression_df.index.isin(test_indices)]

In [37]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('\nCUDA device: GPU\n')
else:
    device = torch.device('cpu')
    print('\nCUDA device: CPU\n')
gc.collect()
torch.cuda.empty_cache()

# device = "cpu"
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"


CUDA device: GPU



In [6]:
torch.cuda.mem_get_info()[0]/1024/1024

23997.6875

In [38]:
class GenoDataset(Dataset):
    def __init__(self, seq_df, transform, dataset_path, seq_len, max_augm_shift=0, 
                 mode='train', regression=False):

        if dataset_path.endswith('.fa'):
            self.fasta = pysam.FastaFile(dataset_path)
        else:
            self.fasta = None

        self.seq_df = seq_df
        self.transform = transform
        self.max_augm_shift = max_augm_shift
        self.mode = mode
        self.seq_len = seq_len

        self.regression = regression

    def __len__(self):
        return len(self.seq_df)

    def __getitem__(self, idx):
        if self.fasta:
            seq = self.fasta.fetch(self.seq_df.iloc[idx].seq_name).upper()
        else:
            seq = self.seq_df.iloc[idx].seq.upper()
        shift = np.random.randint(self.max_augm_shift+1) #random shift at training, must be chunk_size-input_params.seq_len
        seq = seq[shift:shift+self.seq_len] #shift the sequence and limit its size
        seg_label = self.seq_df.iloc[idx].seg_label #label for segment-aware training

        #for given genotype, randomly choose a haplotype for training/testing
        if np.random.rand()>0.5:
            seq = seq.replace('-','').replace('B','A').replace('F','A').replace('M','R')
        else:
            seq = seq.replace('-','').replace('B','A').replace('M','A').replace('F','R')

        masked_sequence, target_labels_masked, target_labels, _, _ = self.transform(seq)
        masked_sequence = (masked_sequence, seg_label)

        if self.regression: 
            seq_expr = self.seq_df.iloc[idx].expr.astype(np.float32)
            return masked_sequence, target_labels_masked, target_labels, seq, seq_expr

        return masked_sequence, target_labels_masked, target_labels, seq

    def close(self):
        self.fasta.close()


class HaploDataset(Dataset):

    def __init__(self, seq_df, transform, dataset_path, seq_len, max_augm_shift=0, 
                 mode='train', regression=False):

        if dataset_path.endswith('.fa'):
            self.fasta = pysam.FastaFile(dataset_path)
        else:
            self.fasta = None

        self.seq_df = seq_df
        self.transform = transform
        self.max_augm_shift = max_augm_shift
        self.mode = mode
        self.seq_len = seq_len

        self.regression = regression

    def __len__(self):

        return len(self.seq_df) # times two because returns both haplotypes 

    def __getitem__(self, idx):

        if self.fasta:
            seq = self.fasta.fetch(self.seq_df.iloc[idx].seq_name).upper()
        else:
            seq = self.seq_df.iloc[idx].seq.upper()

        shift = np.random.randint(self.max_augm_shift+1) #random shift at training, must be chunk_size-input_params.seq_len

        seq = seq[shift:shift+self.seq_len] #shift the sequence and limit its size
        seg_label = self.seq_df.iloc[idx].seg_label #label for segment-aware training
        #'''
        seq1 = seq.replace('-','').replace('B','A').replace('F','A').replace('M','R') # father
        seq2 = seq.replace('-','').replace('B','A').replace('M','A').replace('F','R') # mother 

        masked_sequence1, target_labels_masked1, target_labels1, _, _ = self.transform(seq1)
        masked_sequence2, target_labels_masked2, target_labels2, _, _ = self.transform(seq2)

        masked_sequence = torch.vstack((masked_sequence1, masked_sequence2))
        seg_label = torch.vstack((torch.tensor(seg_label), torch.tensor(seg_label)))
        masked_sequence = (masked_sequence, seg_label)

        target_labels_masked = torch.vstack((target_labels_masked1, target_labels_masked2))
        target_labels = torch.vstack((target_labels1, target_labels2))
        seq = (seq1, seq2)

        if self.regression: 
            seq_expr = self.seq_df.iloc[idx].expr.astype(np.float32)
            return masked_sequence, target_labels_masked, target_labels, seq, seq_expr

        return masked_sequence, target_labels_masked, target_labels, seq
        

    def close(self):
        self.fasta.close()

class ExpressionCollator(object): 
    def __init__(self, haplotypes=False): 
        self.haplotypes = haplotypes

    def __call__(self, data): 
        return self.collate_fn(data)

    def collate_fn(self, data):
        """collate fn that adds expression values for each sequence.
        """ 
        #masked sequence
        if self.haplotypes:
            masked_sequence = [x[0][0] for x in data]
            masked_sequence = [torch.stack(torch.split(d, 3)) for d in masked_sequence] 
            masked_sequence = torch.concat(masked_sequence)
            #seg labels
            seg_labels = [x[0][1] for x in data]
            seg_labels = torch.concat(seg_labels).flatten()
            # target labels masked
            target_labels_masked = [x[1] for x in data]
            target_labels_masked = torch.concat(target_labels_masked)
            # target labels 
            target_labels = [x[2] for x in data]
            target_labels = torch.concat(target_labels)
            #seq
            seqs = [x[3] for x in data]
            seqs = tuple(chain.from_iterable(seqs))

            seg_expr = [x[4] for x in data]
            # repeat each element twice, once for each haplotype
            seg_expr = torch.Tensor(seg_expr).repeat_interleave(2)

            return (masked_sequence, seg_labels, seg_expr),target_labels_masked, target_labels, seqs
        else: 
            masked_sequence = [x[0][0] for x in data]
            masked_sequence = [torch.stack(torch.split(d, 3)) for d in masked_sequence] 
            masked_sequence = torch.concat(masked_sequence)

            seg_labels = torch.Tensor([x[0][1] for x in data]).type(torch.LongTensor)

            target_labels_masked = [x[1] for x in data]
            target_labels_masked = torch.vstack(target_labels_masked)

            target_labels = [x[2] for x in data]
            target_labels = torch.vstack(target_labels)

            seqs = [x[3] for x in data]
            
            seg_expr = torch.Tensor([x[4] for x in data])

            return (masked_sequence, seg_labels, seg_expr), target_labels_masked, target_labels, seqs
    

In [39]:
test_df = None 
if input_params.haplotypes:
    dataset = HaploDataset
    collator = ExpressionCollator(haplotypes=True)
else:
    dataset = GenoDataset
    collator = ExpressionCollator(haplotypes=False)

if not input_params.test: #Train and Validate
    seq_transform = sequence_encoders.SequenceDataEncoder(seq_len = input_params.seq_len, total_len = input_params.seq_len,
                                                      mask_rate = input_params.mask_rate, split_mask = input_params.split_mask)

    #N_train = int(len(seq_expression_df)*(1-input_params.val_fraction))
    if input_params.fold is not None:
        
        samples = seq_expression_df.sample_id.unique()
        val_samples = samples[input_params.fold::input_params.Nfolds] 
        train_df = seq_expression_df[~seq_expression_df.sample_id.isin(val_samples)] 
        test_df = seq_expression_df[seq_expression_df.sample_id.isin(val_samples)]
        test_dataset = dataset(test_df, transform = seq_transform, dataset_path=input_params.dataset, seq_len=input_params.seq_len, mode='eval', regression=True)
        test_dataloader = DataLoader(dataset = test_dataset, batch_size = input_params.batch_size, num_workers = 0, collate_fn = collator, shuffle = False)
    else:
        train_df = seq_expression_df
        #train_df = seq_expression_df[seq_expression_df.split=='train']
        #test_df = seq_expression_df[seq_expression_df.split=='val']
  
    N_train = len(train_df)
    train_fold = np.repeat(list(range(input_params.train_splits)),repeats = N_train // input_params.train_splits + 1 )
    train_df['train_fold'] = train_fold[:N_train]
    # create training dataset & dataloader 
    train_dataset = dataset(train_df, transform = seq_transform, dataset_path=input_params.dataset, seq_len=input_params.seq_len,  mode='train', regression=True)
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = input_params.batch_size, num_workers = 2, collate_fn = collator, shuffle = False)

elif input_params.get_embeddings:
    if input_params.mask_at_test:
        seq_transform = sequence_encoders.RollingMasker(mask_stride = 50, frame = 0)
    else:
        seq_transform = sequence_encoders.PlainOneHot(frame = 0, padding = 'none')
    # create test dataset & dataloader 
    test_dataset = dataset(seq_expression_df, transform = seq_transform, mode='eval', dataset_path=input_params.dataset, seq_len=input_params.seq_len, regression=True)
    test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, num_workers = 1, collate_fn = collator, shuffle = False)

else: #Test
    print("not getting embeddings")
    seq_transform = sequence_encoders.SequenceDataEncoder(seq_len = input_params.seq_len, total_len = input_params.seq_len,
                                                      mask_rate=input_params.mask_rate, split_mask = input_params.split_mask)
    # create test dataset & dataloader 
    test_dataset = dataset(seq_expression_df, transform = seq_transform, mode='eval', dataset_path=input_params.dataset, seq_len=input_params.seq_len, regression=True)
    test_dataloader = DataLoader(dataset = test_dataset, batch_size = input_params.batch_size, num_workers = 2, collate_fn = collator, shuffle = False)

/scratch/tmp/pjo/ipykernel_3967143/2555347109.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['train_fold'] = train_fold[:N_train]


In [40]:
seg_encoder = SpecAdd(embed = True, encoder = 'label', Nsegments=seq_df.seg_label.nunique(), d_model = input_params.d_model)

model = DSSResNetExpression(d_input = 3, d_output = 3, d_model = input_params.d_model, n_layers = input_params.n_layers, 
                     dropout = input_params.dropout, embed_before = True, species_encoder = seg_encoder)

model = model.to(device)

# load the weights
# model.load_state_dict(torch.load(input_params.model_weight, map_location=device), strict=False)

# define which layers to freeze
for param_idx, (param_name, param) in enumerate(model.named_parameters()): 
    if not param_name.startswith("regression"):
        param.requires_grad = False

model_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(model_params, lr = input_params.learning_rate, weight_decay = input_params.weight_decay)

weights_dir = os.path.join(input_params.output_dir, 'weights') #dir to save model weights at save_at epochs

if input_params.model_weight:
    if torch.cuda.is_available():
        #load on gpu
        model.load_state_dict(torch.load(input_params.model_weight), strict=False)
        if input_params.optimizer_weight:
            optimizer.load_state_dict(torch.load(input_params.optimizer_weight))
    else:
        #load on cpu
        model.load_state_dict(torch.load(input_params.model_weight, map_location=torch.device('cpu')), strict=False)
        if input_params.optimizer_weight:
            optimizer.load_state_dict(torch.load(input_params.optimizer_weight, map_location=torch.device('cpu')))


In [154]:
def metrics_to_str(metrics):
    expr_loss, loss, accuracy, masked_acc, masked_recall, masked_IQS = metrics
    return f'expr_loss: {expr_loss:.4}, loss: {loss:.4}, acc: {accuracy:.4}, masked acc: {masked_acc:.4}, {misc.print_class_recall(masked_recall, "masked recall: ")}, masked IQS: {masked_IQS:.4}'

In [232]:
from helpers.metrics import MeanRecall, MaskedAccuracy, IQS
from helpers.misc import EMA, print_class_recall
from tqdm import tqdm


def print_gpu_mb(device="cuda"): 
    if (device == "cuda" or device == torch.device(type="cuda")) and torch.cuda.is_available(): 
        used = torch.cuda.memory_allocated(device)/1024/1024
        return f"{used:.2f} Mb"
    else: 
        return "N/A"

def train_reg_model(model, optimizer, dataloader, device, silent=False, log_wandb=True):
    """train function with added regression loss"""

    criterion = torch.nn.CrossEntropyLoss(reduction = "mean")

    accuracy = MaskedAccuracy(smooth=True).to(device)
    masked_recall = MeanRecall().to(device)
    masked_accuracy = MaskedAccuracy(smooth=True).to(device)
    masked_IQS = IQS().to(device)

    reg_criterion = nn.MSELoss()
    
    model.train() #model to train mode

    if not silent:
        tot_itr = len(dataloader.dataset)//dataloader.batch_size #total train iterations
        pbar = tqdm(total = tot_itr, ncols=750) #progress bar

    loss_EMA = EMA()

    for itr_idx, ((masked_sequence, species_label, expr), targets_masked, targets, _) in enumerate(dataloader):

        masked_sequence = masked_sequence.to(device)
        species_label = species_label.to(device)
        targets_masked = targets_masked.to(device)
        targets = targets.to(device)
        expr = expr.reshape(-1, 1).to(device)

        logits, _, expr_pred = model(masked_sequence, species_label)

        masked_loss = criterion(logits, targets_masked)
        expr_loss = reg_criterion(expr_pred, expr)

        optimizer.zero_grad()

        loss = expr_loss + masked_loss

        loss.backward()

        #if max_abs_grad:
        #    torch.nn.utils.clip_grad_value_(model.parameters(), max_abs_grad)

        optimizer.step()

        if log_wandb:
            if itr_idx % 10 == 0:
                wandb.log({"expr_loss": expr_loss, "masked_loss": masked_loss, "total_loss": loss,
                        "masked_acc": masked_accuracy.compute(), "masked_recall": masked_recall.compute(), "masked_IQS": masked_IQS.compute()})

        smoothed_loss = loss_EMA.update(loss.item())

        preds = torch.argmax(logits, dim=1)

        accuracy.update(preds, targets)
        masked_recall.update(preds, targets_masked)
        masked_accuracy.update(preds, targets_masked)
        masked_IQS.update(preds, targets_masked)
        
        if not silent:
            pbar.update(1)
            pbar.set_description(f'Mem: {print_gpu_mb(device)}, reg_loss: {expr_loss:.4f}, masked_loss: {masked_loss}, total_loss: {smoothed_loss:.4f}, acc: {accuracy.compute():.4}, {print_class_recall(masked_recall.compute(), "masked recall: ")}, masked acc: {masked_accuracy.compute():.4}, masked IQS: {masked_IQS.compute():.4}')

    if not silent:
        pbar.reset()
        del pbar

    return expr_loss, loss, accuracy, masked_accuracy, masked_recall, masked_IQS


In [169]:
check_batch = (
    train_dataset[0], 
    train_dataset[1]
)

In [170]:
(masked_sequence, species_label), targets_masked, targets, _, expr = check_batch

ValueError: not enough values to unpack (expected 5, got 2)

In [168]:
expr

862.9219

In [172]:
check_batch = next(iter(train_dataloader))

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/scratch/tmp/pjo/ipykernel_3958059/4106035108.py", line 110, in __call__
    self.collate_fn(data)
  File "/scratch/tmp/pjo/ipykernel_3958059/4106035108.py", line 139, in collate_fn
    (masked_sequence, species_label), targets_masked, targets, seq, seg_expr = data
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 5)


In [156]:
(masked_sequence, species_label, expr), targets_masked, targets, _ = check_batch

TypeError: cannot unpack non-iterable ExpressionCollator object

In [13]:
len(optimizer.param_groups[0]["params"])

6

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jokpro (cmm-t1). Use `wandb login --relogin` to force relogin


In [15]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [233]:
from helpers import misc
import wandb

from IPython.display import clear_output
last_epoch = 0

log_wandb = True

clear_output()

with open("train_dataloader.pkl", "wb") as f:
    pickle.dump(train_dataloader, f)
with open("test_dataloader.pkl", "wb") as f:
    pickle.dump(test_dataloader, f)

#from helpers.misc import print    #print function that displays time
if log_wandb:
    wandb.init(input_params.wandb_proj, input_params.run_name, id=input_params.run_name) # seq_len, batch_size, geno or haplo, loss (full = reg + masked)
    wandb.watch(model, log_freq=100)

if not input_params.test:

    print("Total samples in train set: ", len(train_df))

    for epoch in range(last_epoch+1, input_params.tot_epochs+1):

        print(f'EPOCH {epoch}: Training...')

        #if input_params.masking == 'stratified_maf':

        #    meta = get_random_mask()

        train_dataset.seq_df = train_df[train_df.train_fold == (epoch-1) % input_params.train_splits]

        train_metrics = train_reg_model(model, optimizer, train_dataloader, device,
                            silent = False, log_wandb=log_wandb)
            
        # print(f'epoch {epoch} - train, {metrics_to_str(train_metrics)}')

        if epoch in input_params.save_at: #save model weights

            misc.save_model_weights(model, optimizer, weights_dir, epoch)

        # if test_df is not None  and ( epoch==input_params.tot_epochs or
        #                     (input_params.validate_every and epoch%input_params.validate_every==0)):

        #     print(f'EPOCH {epoch}: Validating...')

        #     val_metrics, *_ =  train_eval.model_eval(model, optimizer, test_dataloader, device,
        #             silent = False)

        #     print(f'epoch {epoch} - validation, {metrics_to_str(val_metrics)}')
            
        #lr_scheduler.step()
else:

    print(f'EPOCH {last_epoch}: Test/Inference...')

    test_metrics, test_embeddings, motif_probas =  train_eval.model_eval(model, test_dataloader, device, 
                                                          get_embeddings = input_params.get_embeddings, diploid=input_params.diploid,
                                                          silent = False)
    
    

    print(f'epoch {last_epoch} - test, {metrics_to_str(test_metrics)}')

    if input_params.get_embeddings:
        
        os.makedirs(input_params.output_dir, exist_ok = True)

        with open(input_params.output_dir + '/embeddings.pickle', 'wb') as f:
            #test_embeddings = np.vstack(test_embeddings)
            #np.save(f,test_embeddings)
            pickle.dump(test_embeddings,f)
            #pickle.dump(seq_df.seq_name.tolist(),f)
            
print()
print(f'peak GPU memory allocation: {round(torch.cuda.max_memory_allocated(device)/1024/1024)} Mb')
print('Done')

Total samples in train set:  5130
EPOCH 1: Training...


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0/641 [00:00<?, ?it/s]/data/nasif12/home_if12/pjo/Documents/lm-eqtl/model/my_notebooks/../models/dss.py:335: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /opt/conda/conda-bld/pytorch_1695392026823/work/aten/src/ATen/nati

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/traitlets/traitlets.py", line 653, in get
    value = obj._trait_values[self.name]
            ~~~~~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: 'layout'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/wandb/__main__.py", line 3, in <module>
    cli.cli(prog_name="python -m wandb")
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/modules/i12g/anaconda/envs/pjo_dnareg/lib/python3.11/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/

In [ ]:
torch.functional.F.embedding()

In [113]:
test_batch = next(iter(train_dataloader))

In [217]:
(masked_sequence, species_label, expr), targets_masked, targets, _ = check_batch
species_label

tensor([7., 6., 6., 0., 9., 9., 2., 2.])

In [1]:
print_gpu_mb()


NameError: name 'print_gpu_mb' is not defined

In [48]:
print_gpu_mb()

'22219.69 Mb (91.59%)'

In [40]:
torch.cuda.memory_stats(device="cpu")

ValueError: Expected a cuda device, but got: cpu

In [37]:
torch.cuda.memory_reserved()/1024/1024

1708.0

In [35]:
torch.cuda.mem_get_info()

(23299031040, 25438126080)

In [66]:
with open("debug.pkl", "rb") as f: 
    debug_data = pickle.load(f)
    pre_encoder = debug_data["pre_encoder"]
    post_encoder = debug_data["post_encoder"]
    encoder = debug_data["encoder"]

with open("debug_x.pkl", "rb") as f: 
    test_x = pickle.load(f)

In [62]:
random_x = torch.Tensor(np.random.randint(0, 2, size=pre_encoder.shape))


In [63]:
random_x.shape

torch.Size([8, 3, 500])

In [67]:
encoder(test_x)

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]

In [69]:
encoder

Parameter containing:
tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], requires_grad=True)

In [9]:
from torch.functional import F

In [22]:
weights = torch.randn(256, 3, 5)
F.conv1d(debug_x, weights, padding=int((15-1)/2)).shape

torch.Size([16, 256, 510])

In [24]:

conv = nn.Conv1d(in_channels=3,out_channels=256,padding=int((15-1)/2),kernel_size=15,bias=True)

In [21]:
torch.randn(256, 16, 5).shape

torch.Size([256, 16, 5])

In [27]:
conv(debug_x)

tensor([[[-0.2720, -0.0660,  0.0992,  ...,  0.0964,  0.0253,  0.0646],
         [-0.2971, -0.0209, -0.0725,  ..., -0.3556, -0.1917,  0.2895],
         [ 0.0270,  0.4130,  0.6566,  ..., -0.1245,  0.0718, -0.0956],
         ...,
         [-0.3847, -0.7098, -0.5146,  ...,  0.3219, -0.2450, -0.2260],
         [ 0.2712,  0.2367,  0.0239,  ...,  0.1905,  0.0729, -0.2356],
         [-0.0473,  0.0387, -0.3265,  ..., -0.0301,  0.1991, -0.1900]],

        [[-0.2720, -0.0660,  0.0992,  ...,  0.0964,  0.0253,  0.0646],
         [-0.2971, -0.0209, -0.0725,  ..., -0.3556, -0.1917,  0.2895],
         [ 0.0270,  0.4130,  0.6566,  ..., -0.1245,  0.0718, -0.0956],
         ...,
         [-0.3847, -0.7098, -0.5146,  ...,  0.3219, -0.2450, -0.2260],
         [ 0.2712,  0.2367,  0.0239,  ...,  0.1905,  0.0729, -0.2356],
         [-0.0473,  0.0387, -0.3265,  ..., -0.0301,  0.1991, -0.1900]],

        [[-0.0770, -0.0696,  0.0272,  ..., -0.0732,  0.0342,  0.0115],
         [-0.0332, -0.0268,  0.0296,  ...,  0

In [ ]:
last_epoch = 0

if input_params.model_weight:
    print("With loaded model weights:")

    if torch.cuda.is_available():
        print("Using GPU...")
        #load on gpu
        model.load_state_dict(torch.load(input_params.model_weight))
        if input_params.optimizer_weight:
            optimizer.load_state_dict(torch.load(input_params.optimizer_weight))
    else:
        #load on cpu
        model.load_state_dict(torch.load(input_params.model_weight, map_location=torch.device('cpu')))
        if input_params.optimizer_weight:
            optimizer.load_state_dict(torch.load(input_params.optimizer_weight, map_location=torch.device('cpu')))

    last_epoch = int(input_params.model_weight.split('_')[-3]) #infer previous epoch from input_params.model_weight

weights_dir = os.path.join(input_params.output_dir, 'weights') #dir to save model weights at save_at epochs

if input_params.save_at:
    os.makedirs(weights_dir, exist_ok = True)

#lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
#        milestones=input_params.lr_sch_milestones, gamma=input_params.lr_sch_gamma, verbose=False) 

With loaded model weights:
Using GPU...


RuntimeError: Error(s) in loading state_dict for DSSResNetExpression:
	Missing key(s) in state_dict: "regression_head.0.weight", "regression_head.0.bias", "regression_head.2.weight", "regression_head.2.bias", "regression_head.4.weight", "regression_head.4.bias". 